# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [1]:
from mxnet import ndarray as nd
import mxnet as mx

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

### Mxnet

In [2]:
shape = (4096, 4096)
A = nd.random_normal(shape=shape)
B = nd.random_normal(shape=shape)

In [3]:
import time
tic = time.time()
C = nd.dot(A,B)
C.wait_to_read(); toc = time.time()
print('vectorized multiplication: computed in %fs' % (toc - tic))

vectorized multiplication: computed in 1.099711s


In [4]:
C_1 = nd.zeros_like(A)
tic = time.time()
for i in range(B.shape[0]):
    C_1[:,i] = nd.dot(A, B[:,i])
C_1.wait_to_read()
toc = time.time()
print('semi vectorized multiplication: computed in %fs' %(toc - tic))

semi vectorized multiplication: computed in 31.661144s


### Pytorch

In [5]:
import torch
import time

In [6]:
shape = (4096, 4096)

In [7]:
A = torch.normal(0, 1, shape)
B = torch.normal(0, 1, shape)

In [8]:
tic = time.time()
C = A.mm(B)
toc = time.time()
print('vectorized multiplication: computed in {:.6f}'.format(toc-tic))

vectorized multiplication: computed in 0.568102


In [9]:
tic = time.time()
C_1 = torch.zeros_like(A)
for i in range(B.shape[1]):
    C_1[:,i] = A.mv(B[:,i])
toc = time.time()
print('semi vectorized multiplication: computed in {:.6f}'.format(toc-tic))

semi vectorized multiplication: computed in 14.131060


## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

In [10]:
from torch.distributions import uniform

In [11]:
shape = (10, 9)
A = torch.randn(shape)
D = torch.diag(uniform.Uniform(0, 100).sample((shape[1], )))

In [12]:
B = (A@D)@A.T

B is symmetric matrix

In [13]:
torch.eig(B).eigenvalues >= 0

tensor([[True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True],
        [True, True]])

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [14]:
torch.ones(5).cuda()

tensor([1., 1., 1., 1., 1.], device='cuda:0')

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [16]:
import numpy as np

In [30]:
shape = (4096, 4096)
A = torch.randn(shape)
B = torch.randn(shape)
c = np.zeros((shape[0],))
tic = time.time()
for i in range(B.shape[0]):
    c[i] = ((A.mv(B[:,i])).norm())**2
toc = time.time()
print("time %fs" %(toc-tic))

time 12.573429s


In [31]:
d = torch.empty((shape[0], ))
tic = time.time()
for i in range(B.shape[0]):
    d[i] = ((A.mv(B[:,i])).norm())**2
c = d.numpy()
toc = time.time()
print("time %fs" %(toc-tic))

time 12.478082s


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [33]:
C = torch.rand_like(A)
tic = time.time()
C += A.mm(B)
toc = time.time()
print("time %fs" %(toc-tic))

time 0.548046s


## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [44]:
x = torch.linspace(-10, 10, 5)
j = torch.arange(1, 5, 1).view(1, -1)
A = (x.view(-1,1))**j

In [46]:
A, j

(tensor([[-1.0000e+01,  1.0000e+02, -1.0000e+03,  1.0000e+04],
         [-5.0000e+00,  2.5000e+01, -1.2500e+02,  6.2500e+02],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [ 5.0000e+00,  2.5000e+01,  1.2500e+02,  6.2500e+02],
         [ 1.0000e+01,  1.0000e+02,  1.0000e+03,  1.0000e+04]]),
 tensor([[1, 2, 3, 4]]))